In [3]:
import pandas as pd
from helpers import read_csv, load_pickled_data, pickle_data
import os
import matplotlib.pyplot as plt 
import plotly
from helpers import create_nx_graph
import plotly.graph_objects as go
import os, sys

from helpers import bfs, get_class_pairs, choose_node_based_on_neighbors, load_pickled_data
import numpy as np
import networkx as nx

importances_dict:dict = load_pickled_data("feature_importances.pkl")
feat_occurences:dict = load_pickled_data('frequencies.pkl')
print(feat_occurences)
importances = {}
names = []
i = 0
for key, value in importances_dict.items():
    importances[i] = value
    names.append(key)


keep_cols = ["txId", 'Time step'] + names
TIMESTEPS =[13, 42, 35, 32, 29, 22, 9 ,20, 16, 15, 17, 20, 21, 24, 25, 26, 29, 31, 38, 40, 41, 42]

DATASET_BASE = "dataset"
PICKLED_TIMESTEPS = os.path.join("exports", "split_on_timestep.pkl")
txs_features:pd.DataFrame =  read_csv(os.path.join(DATASET_BASE, "elliptic_txs_features.csv"), ret_Dataframe=True)[keep_cols]



{'Local_feature_2': {46: 1, 45: 1, 15: 330, 25: 238, 44: 18, 28: 86, 34: 189, 21: 37, 30: 925, 38: 12, 37: 572, 14: 736, 36: 119, 8: 70, 23: 214, 32: 26, 29: 112, 13: 19, 39: 2, 42: 4, 18: 329, 6: 13, 43: 41, 5: 175, 26: 54, 33: 20, 9: 31, 31: 18, 4: 5, 3: 18, 24: 7, 41: 6, 35: 3, 16: 22, 7: 18, 27: 7, 20: 3, 11: 8, 10: 11, 40: 1, 12: 6, 1: 7, 2: 1, 22: 1}, 'Local_feature_5': {13: 14, 1: 3315, 2: 293, 0: 677, 4: 145, 10: 20, 9: 16, 6: 8, 12: 3, 5: 13, 8: 6, 7: 6}, 'Local_feature_18': {8: 82, 5: 27, 14: 212, 19: 708, 9: 89, 1: 694, 3: 42, 28: 830, 23: 385, 38: 86, 40: 101, 21: 338, 34: 22, 36: 19, 26: 72, 50: 35, 13: 64, 45: 11, 22: 154, 2: 1, 20: 138, 6: 12, 49: 8, 32: 9, 41: 24, 25: 59, 52: 20, 31: 10, 12: 32, 35: 47, 39: 15, 42: 12, 15: 25, 51: 15, 10: 31, 4: 15, 46: 24, 37: 3, 7: 6, 47: 8, 48: 1, 11: 4, 18: 1, 33: 2, 53: 4, 44: 8, 54: 1, 17: 1, 24: 1, 29: 1, 30: 3, 43: 2, 16: 1, 27: 1}, 'Local_feature_41': {0: 2480, 21: 453, 17: 277, 16: 97, 19: 151, 5: 77, 9: 36, 28: 315, 33: 82, 1

In [4]:
txs_features = txs_features[keep_cols]
txs_classes:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_classes.csv"), ret_Dataframe=True)
txs_edgelist:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_edgelist.csv"), ret_Dataframe=True)


merge_feats_class = pd.merge(txs_features, txs_classes, on='txId').dropna()
merge_feats_class.shape
print(f"Total Nodes df shape: {txs_features.shape}")
print(f"total edges df shape: {txs_edgelist.shape}")
#timestep_graphs = load_pickled_data(PICKLED_TIMESTEPS)
timestep_dataframe = merge_feats_class.loc[merge_feats_class['Time step'].isin(TIMESTEPS)].reset_index(drop=True)
timestep_edges = txs_edgelist[txs_edgelist['txId1'].isin(timestep_dataframe['txId'])].reset_index(drop=True)

from helpers import discretize_per_column
freq, timestep_dataframe = discretize_per_column(merge_feats_class, 7, 0, True)
timestep_cols = timestep_dataframe.columns
timestep_list = timestep_dataframe.values.tolist()

edge_list = timestep_edges.values.tolist()

from Graph import Graph, Node
from visualizations import visualize_graph_from_list, visualize_graph_df

graph_of_interest = Graph()
graph_of_interest.create_from_lists(timestep_cols, timestep_list, edge_list)
graph_of_interest.feature_importances =  importances_dict
graph_of_interest.feature_occurences = freq
pickle_data("working_graph3.pkl", graph_of_interest, True)

# g:nx.DiGraph = create_nx_graph(timestep_dataframe, timestep_edges)

Total Nodes df shape: (203769, 31)
total edges df shape: (234355, 2)
Local_feature_2
{46: 33350, 0: 67, 97: 1539, 245: 665, 118: 10673, 274: 64, 157: 1096, 220: 6650, 394: 195, 3: 1431, 8: 1495, 260: 9998, 103: 515, 229: 477, 115: 1212, 171: 7131, 261: 505, 406: 1284, 414: 527, 297: 1014, 98: 526, 87: 1812, 23: 88, 131: 94, 74: 33, 170: 1492, 410: 471, 417: 153, 86: 1503, 298: 435, 281: 299, 24: 374, 218: 713, 52: 142, 55: 1500, 21: 269, 197: 417, 327: 164, 132: 784, 12: 686, 48: 1289, 5: 274, 56: 106, 72: 125, 13: 229, 6: 96, 57: 135, 45: 604, 15: 153, 94: 499, 246: 93, 50: 100, 136: 376, 369: 2055, 361: 1860, 1: 556, 181: 216, 232: 457, 75: 183, 415: 1027, 262: 84, 276: 1358, 137: 203, 38: 514, 47: 28, 172: 537, 332: 1715, 54: 244, 402: 1093, 185: 924, 102: 39, 285: 77, 387: 1470, 113: 483, 88: 1483, 124: 285, 40: 148, 105: 1360, 179: 167, 355: 796, 305: 736, 31: 31, 20: 2182, 83: 206, 7: 169, 77: 20, 84: 122, 32: 19, 219: 336, 142: 206, 96: 207, 10: 59, 43: 163, 44: 646, 350: 275, 7

In [5]:
timestep_dataframe

,txId,Time step,Local_feature_2,Local_feature_5,Local_feature_18,Local_feature_41,Local_feature_47,Local_feature_52,Local_feature_53,Local_feature_55,...,Local_feature_90,Local_feature_91,Aggregate_feature_43,Aggregate_feature_44,Aggregate_feature_49,Aggregate_feature_52,fees,size,num_output_addresses,class
0,3321,1,46,1,16,10,11,201,329,269,...,38,96,8,7,68,46,46,9,2,3
1,11108,1,46,1,19,10,9,59,188,122,...,26,73,3,7,34,58,46,9,2,3
2,51816,1,46,1,3,0,1,12,28,27,...,38,96,1,7,34,18,46,9,2,3
3,68869,1,46,1,21,10,9,201,326,269,...,37,96,3,7,34,18,46,61,2,2
4,89273,1,0,23,18,80,68,262,399,342,...,37,117,1,247,0,85,1,101,23,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,3,1,49,0,1,4,3,4,...,14,53,223,209,0,53,3,8,2,3
202800,194747925,49,3,1,49,0,1,4,3,4,...,21,65,223,210,0,58,3,8,2,3
202801,194748063,49,3,1,49,0,1,4,3,4,...,26,73,223,206,0,120,3,9,2,3
202802,194748070,49,3,1,49,0,1,4,3,4,...,26,73,223,209,0,58,3,9,2,3


In [6]:
graph_dict = {}
for timestep in set(timestep_dataframe["Time step"].values):
    single_timestep_dataframe = timestep_dataframe.loc[timestep_dataframe['Time step'].isin([timestep])].reset_index(drop=True)
    single_timestep_edges = timestep_edges[timestep_edges['txId1'].isin(single_timestep_dataframe['txId'])].reset_index(drop=True)
    t_cols = single_timestep_dataframe.columns
    t_list = single_timestep_dataframe.values.tolist()
    e_list = single_timestep_edges.values.tolist()
    print(len(t_list))
    g = Graph()
    g.create_from_lists(t_cols, t_list, e_list)
    g.feature_importances =  importances_dict
    g.feature_occurences = freq
    graph_dict[timestep] = g
    
pickle_data("working_graphs_dict3.pkl", graph_dict, True)


7844
4529
6621
5689
6801
4328
6026
4431
4991
6727
4289
2039
4516
2020
3623
2963
3373
1969
3506
4287
3534
5878
4132
4580
2307
2519
1077
1652
4266
2475
2775
4513
3148
2436
5486
6276
3244
2874
2717
4458
5312
7125
4919
4952
5558
3512
5105
2954
2448
Succesfully pickled the object on file working_graphs_dict.pkl


In [ ]:
print(set(timestep_dataframe["Time step"].values))


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49}


NameError: name 'freq' is not defined